In [1]:
import math
import numpy as np
import random
import itertools as its
import more_itertools as mits
import copy
from operator import itemgetter
import multiprocessing
import time

In [2]:
# Random seed
random.seed(42)
# Do extra printing and counting
verbose = True
# CPU count for multiprocesing
cpuCount = multiprocessing.cpu_count()
# Mask length
maskLen = 10
# Number of solutions to have
n = 500
# Top 20%
s = math.ceil(n * 0.67)
# Number of masks in solution
l = 44
# Number of off target combinations
goal = 4845

In [3]:
# Generate off target masks
def genOfftargets():
    combinations = [
        # [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
        # [0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
        # [0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
        [0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
    ]
    offTargets = np.zeros((4845,20), int)
    pos = 0
    
    for combo in combinations:
        for p in mits.distinct_permutations(combo):
            offTargets[pos,:] = p
            pos += 1

    return offTargets

# The off target combinations
ot = genOfftargets()

In [4]:
# Draw function. Randomly generates mask
# Where k is the number of ones in the mask
def drawMask():
    global maskLen
    newMask = np.zeros(20, dtype=int)
    for pos in random.sample(range(20), k=maskLen):
        newMask[pos] = 1
    return newMask

In [5]:
# Generate solution. A solution is a set of masks
def genSolution():
    global l
    newSoltuion = np.zeros((l, 20), dtype=int)
    for i in range(l):
        newSoltuion[i,:] = drawMask()
    return newSoltuion

In [6]:
# Score mask
def scoreMask(mask):
    global ot
    matchCountColumn = np.matmul(ot,mask)
    detectionColumn = np.where(matchCountColumn < maskLen, 0, 1)
    return detectionColumn

In [7]:
# Score mask
def scoreMaskNew(mask):
    global ot
    mismatchMatrix = np.bitwise_and(mask, ot)
    matchCountColumn = mismatchMatrix.sum(axis=1)
    detectedCountColumn = np.where(matchCountColumn < maskLen, 0, 1)
    return (detectedCountColumn, matchCountColumn, mismatchMatrix)

In [8]:
# Score solution
def scoreSolution(solution):
    global ot
    detectionMatrix = np.zeros((len(ot),len(solution)), dtype=int)
    for pos, mask in enumerate(solution):
        detectionMatrix[:,pos] = scoreMask(mask)
    score = np.where(detectionMatrix.sum(axis=1) > 0, 1, 0).sum(axis=0)
    return (score, detectionMatrix, solution)

In [9]:
# Score solution
def scoreSolutionNew(maskList):
    global ot
    detectedMatrix = np.zeros((len(ot),len(maskList)), dtype=int)
    detectedCountMatrix = np.zeros((len(ot),1), dtype=int)
    matchCountMatrix =  np.zeros((len(ot),len(maskList)), dtype=int)
    mismatchMatrixList = list(0 for i in range(len(maskList)))
    for pos, mask in enumerate(maskList):
        (detectedCountColumn, matchCountColumn, mismatchMatrix) = scoreMask(mask)
        detectedMatrix[:,pos] = detectedCountColumn
        matchCountMatrix[:,pos] = matchCountColumn
        mismatchMatrixList[pos] = mismatchMatrix
    detectedCountMatrix = detectedMatrix.sum(axis=1)
    score = np.where(detectedCountMatrix > 0, 1, 0).sum(axis=0)
    return (score, maskList, detectedCountMatrix, detectedMatrix, matchCountMatrix, mismatchMatrixList)

In [10]:
# Do the minimal amount of work to update scored solution
def updateScore(scoredSolutionTuple, maskIndex, bitPosition, bitValue):
    (score, maskList, detectedCountMatrix, detectedMatrix, matchCountMatrix, mismatchMatrixList) = copy.deepcopy(scoredSolutionTuple)
    # Update mask
    maskList[maskIndex][bitPosition] =  bitValue

    # Update mismatch matrix for selected mask index
    oldMismatchMatrixColumn = copy.deepcopy(mismatchMatrixList[maskIndex][:, bitPosition])
    newMismatchMatrixColumn = np.bitwise_and(bitValue, ot[:, bitPosition])

    # Update locations that have changed
    oldMatchCountMatrixColumn = copy.deepcopy(matchCountMatrix[:, maskIndex])
    newMatchCountMatrixColumn = copy.deepcopy(matchCountMatrix[:, maskIndex])
    oldDetectedMatrixColumn = copy.deepcopy(detectedMatrix[:, maskIndex])
    newDetectedMatrixColumn = copy.deepcopy(detectedMatrix[:, maskIndex])
    oldDetectedCountMatrixColumn = copy.deepcopy(detectedCountMatrix)
    newDetectedCountMatrixColumn = copy.deepcopy(detectedCountMatrix)

    for offTarget in np.where(np.not_equal(oldMismatchMatrixColumn, newMismatchMatrixColumn))[0]:
        difference = oldMismatchMatrixColumn[offTarget] - newMismatchMatrixColumn[offTarget]
        newMatchCountMatrixColumn[offTarget] =  oldMatchCountMatrixColumn[offTarget] - difference
        newDetectedMatrixColumn[offTarget] = 1 if newMatchCountMatrixColumn[offTarget] == 8 else 0
        if newDetectedMatrixColumn[offTarget] == 1 and oldDetectedMatrixColumn[offTarget] == 0:
            newDetectedCountMatrixColumn[offTarget] += 1
        elif newDetectedMatrixColumn[offTarget] == 0 and oldDetectedMatrixColumn[offTarget] == 1:
            newDetectedCountMatrixColumn[offTarget] -= 1
        if newDetectedCountMatrixColumn[offTarget] > 0 and oldDetectedCountMatrixColumn[offTarget] < 1:
            score += 1
        elif newDetectedCountMatrixColumn[offTarget] < 1 and oldDetectedCountMatrixColumn[offTarget] > 0:
            score -= 1

    mismatchMatrixList[maskIndex][:, bitPosition] = newMismatchMatrixColumn
    matchCountMatrix[:,maskIndex] = newMatchCountMatrixColumn
    detectedMatrix[:, maskIndex] = newDetectedMatrixColumn
    detectedCountMatrix = newDetectedCountMatrixColumn

    return (score, maskList, detectedCountMatrix, detectedMatrix, matchCountMatrix, mismatchMatrixList)


In [11]:
# Mutate function, change whole mask
def mutateRow(scoredSolution):
    global scoredSolutions
    # Copy original
    newSolution = copy.deepcopy(scoredSolution[2])
    newScoreMatrix = copy.deepcopy(scoredSolution[1])
    # Pick random row
    idx = random.randrange(l)
    # Replace with new mask
    newSolution[idx] = drawMask()
    # Update solution
    newScoreMatrix[:,idx] = scoreMask(newSolution[idx])
    # Compare new solution with cut off
    newScore = np.where(newScoreMatrix.sum(axis=1) > 0, 1, 0).sum(axis=0)
    # Return best solution
    if (newScore >= scoredSolutions[s][0]):
        return (newScore, newScoreMatrix, newSolution)
    else:
        return scoredSolution

In [12]:
# Mutate function, change a sinlge element in a row
def mutateElement(scoredSolution):
    global scoredSolutions
    # Copy orignal
    newSolution = copy.deepcopy(scoredSolution[2])
    newScoreMatrix = copy.deepcopy(scoredSolution[1])
    # Pick random row
    idx = random.randrange(l)
    oldMask = copy.deepcopy(newSolution[idx])
    zeros = np.where(oldMask == 0)[0]
    ones = np.where(oldMask == 1)[0]
    zeroToSwap = random.randrange(len(zeros))
    oneToSwap = random.randrange(len(ones))
    # Swap a one with a zero
    ones[oneToSwap] = zeros[zeroToSwap]
    newMask = np.zeros(20,dtype=int)
    for one in ones:
        newMask[one] = 1
    # Update solution
    newSolution[idx] = newMask
    newScoreMatrix[:,idx] = scoreMask(newSolution[idx])
    # Compare new solution with cut off
    newScore = np.where(newScoreMatrix.sum(axis=1) > 0, 1, 0).sum(axis=0)
    # Return best solution
    if (newScore >= scoredSolutions[s][0]):
        return (newScore, newScoreMatrix, newSolution)
    else:
        return scoredSolution

In [13]:
# Mutate function, change a sinlge element in a row and keep trying all possible moves
def mutateElementDefinite(scoredSolution):
    global scoredSolutions
    # Copy orignal
    newSolution = copy.deepcopy(scoredSolution[2])
    newScoreMatrix = copy.deepcopy(scoredSolution[1])
    # Pick random row
    idx = random.randrange(l)
    # Attempt tp mutate mask
    oldMask = copy.deepcopy(newSolution[idx])
    zeros = np.where(oldMask == 0)[0]
    # Jumble ones to randomise order of checking them
    ones = np.random.permutation(np.where(oldMask == 1)[0])
    # Loop through everyone one in the mask and attempt to swap it with every zero until a better score is found
    for oneToSwap in range(len(ones)):
        # Generate a random order for positions to try
        randomPos = np.append(copy.deepcopy(zeros), [ones[oneToSwap]])
        # randomPos = copy.deepcopy(zeros)
        randomPos = np.random.permutation(randomPos)
        for zeroToSwap in range(len(randomPos)):
            updatedOnes = copy.deepcopy(ones)
            # Swap a one with a zero
            updatedOnes[oneToSwap] = randomPos[zeroToSwap]
            # Generate new mask
            newMask = np.zeros(20,dtype=int)
            for one in updatedOnes:
                newMask[one] = 1
            # Update solution
            newSolution[idx] = newMask
            newScoreMatrix[:,idx] = scoreMask(newSolution[idx])
            # Compare new solution with cut off
            newScore = np.where(newScoreMatrix.sum(axis=1) > 0, 1, 0).sum(axis=0)
            # Return early if we fine an appropriate solution
            if (newScore >= scoredSolutions[s][0]):
                return (newScore, newScoreMatrix, newSolution)
    # Default to return old solution
    return scoredSolution

In [14]:
# Mutate every element
def mutateMatrix(scoredSolution):
    global scoredSolutions
    # Copy original
    newSolution = copy.deepcopy(scoredSolution[2])
    newScoreMatrix = copy.deepcopy(scoredSolution[1])
    # Loop every row of solution and do an element switch
    for i in range(len(newSolution)):
        oldMask = copy.deepcopy(newSolution[i])
        zeros = np.where(oldMask == 0)[0]
        ones = np.where(oldMask == 1)[0]
        zeroToSwap = random.randrange(len(zeros))
        oneToSwap = random.randrange(len(ones))
        # Swap a one with a zero
        ones[oneToSwap] = zeros[zeroToSwap]
        newMask = np.zeros(20,dtype=int)
        for one in ones:
            newMask[one] = 1
        # Update solution
        newSolution[i] = newMask
        newScoreMatrix[:,i] = scoreMask(newSolution[i])
    # Compare new solution with cut off
    newScore = np.where(newScoreMatrix.sum(axis=1) > 0, 1, 0).sum(axis=0)
    # Return best solution
    if (newScore >= scoredSolutions[s][0]):
        return (newScore, newScoreMatrix, newSolution)
    else:
        return scoredSolution

In [15]:
# Mutate function, change a sinlge element in a row and keep trying all possible moves
# Should only recalculate single positions that change
def updateMutateElementDefinite(scoredSolution):
    global scoredSolutions
    # Copy original solution
    newScoreSolution = copy.deepcopy(scoredSolution)
    # Pick random row
    idx = random.randrange(l)
    # Copy mask that we are mutating 
    maskToMutate = copy.deepcopy(scoredSolution[1][idx])
    # Get indices where the mask is 0
    zeros = np.where(maskToMutate == 0)[0]
    # Get indices where the mask is 1
    ones = np.where(maskToMutate == 1)[0]
    # Pick random one
    oneToSwap = random.randrange(len(ones))
    # Generate a list of positions to try
    randomPos = np.append(copy.deepcopy(zeros), [ones[oneToSwap]])
    # Jumble up list to try them in a random order
    randomPos = np.random.permutation(randomPos)
    for zeroToSwap in range(len(randomPos)):
        newScoreSolution = updateScore(newScoreSolution, idx, oneToSwap, 0)
        newScoreSolution = updateScore(newScoreSolution, idx, zeroToSwap, 1)
        # Return early if we fine an appropriate solution
        if (newScoreSolution[0] >= scoredSolutions[s][0]):
            return newScoreSolution
    # Default to return original solution
    return scoredSolution


In [16]:
# Mutate function, change a sinlge element in a row and keep trying all possible moves
def solutionMutateElementDefinite(scoredSolution):
    global scoredSolutions
    # Copy orignal
    maskList = copy.deepcopy(scoredSolution[1])
    # Pick random row
    idx = random.randrange(l)
    # Attempt tp mutate mask
    maskToMutate = copy.deepcopy(maskList[idx])
    # Get indices where the mask is 0
    zeros = np.where(maskToMutate == 0)[0]
    # Get indices where the mask is 1
    ones = np.where(maskToMutate == 1)[0]
    # Pick random one
    oneToSwap = random.randrange(len(ones))
    # Generate a list of positions to try
    randomPos = np.append(copy.deepcopy(zeros), [ones[oneToSwap]])
    # Jumble up list to try them in a random order
    randomPos = np.random.permutation(randomPos)
    for zeroToSwap in range(len(randomPos)):
        updatedOnes = copy.deepcopy(ones)
        # Swap a one with a zero
        updatedOnes[oneToSwap] = randomPos[zeroToSwap]
        # Generate new mask
        newMask = np.zeros(20,dtype=int)
        for one in updatedOnes:
            newMask[one] = 1
        # Update solution
        maskList[idx] = newMask
        newScoredSolution = scoreSolution(maskList)
        # Return early if we fine an appropriate solution
        if (newScoredSolution[0] >= scoredSolutions[s][0]):
            return newScoredSolution
    # Default to return old solution
    return scoredSolution

In [17]:
# Mutate function, change a sinlge element in a row and keep trying all possible moves
def maskMutateElementDefinite(scoredSolution):
    global scoredSolutions
    # Copy orignal
    newScoredSolution = copy.deepcopy(scoredSolution)
    # Pick random row
    idx = random.randrange(l)
    # Attempt tp mutate mask
    maskToMutate = copy.deepcopy(newScoredSolution[1][idx])
    # Get indices where the mask is 0
    zeros = np.where(maskToMutate == 0)[0]
    # Get indices where the mask is 1
    ones = np.where(maskToMutate == 1)[0]
    # Pick random one
    oneToSwap = random.randrange(len(ones))
    # Generate a list of positions to try
    randomPos = np.append(copy.deepcopy(zeros), [ones[oneToSwap]])
    # Jumble up list to try them in a random order
    randomPos = np.random.permutation(randomPos)
    for zeroToSwap in range(len(randomPos)):
        updatedOnes = copy.deepcopy(ones)
        # Swap a one with a zero
        updatedOnes[oneToSwap] = randomPos[zeroToSwap]
        # Generate new mask
        newMask = np.zeros(20,dtype=int)
        for one in updatedOnes:
            newMask[one] = 1
        # Update solution
        # return (score, maskList, detectedCountMatrix, detectedMatrix, matchCountMatrix, mismatchMatrixList)
        #
        newScoredSolution[1][idx] = newMask
        (detectedCountColumn, matchCountColumn, mismatchMatrix) = scoreMask(newScoredSolution[1][idx])
        newScoredSolution[3][:,idx] = detectedCountColumn
        newScoredSolution[4][:,idx] = matchCountColumn
        newScoredSolution[5][idx] = mismatchMatrix
        detectedCountMatrix = newScoredSolution[3].sum(axis=1)
        score = np.where(newScoredSolution[2] > 0, 1, 0).sum(axis=0)
        # Return early if we fine an appropriate solution
        if (score >= scoredSolutions[s][0]):
            return (score, newScoredSolution[1], detectedCountMatrix,  newScoredSolution[3], newScoredSolution[4], newScoredSolution[5])
    # Default to return old solution
    return scoredSolution

In [18]:
# Generate combinations from previous solution as starting points
previousSolution = [
    (0, 3, 6, 7, 9, 10, 12, 14, ),
    (4, 6, 7, 8, 12, 15, 17, 19, ),
    (1, 2, 3, 5, 11, 14, 15, 17, ),
    (0, 5, 6, 8, 12, 13, 18, 19, ),
    (0, 3, 7, 11, 14, 16, 17, 18, ),
    (1, 4, 6, 8, 12, 16, 17, 19, ),
    (0, 8, 9, 10, 14, 15, 17, 19, ),
    (1, 6, 9, 10, 12, 13, 15, 17, ),
    (0, 1, 2, 5, 7, 15, 16, 18, ),
    (2, 3, 6, 9, 10, 11, 12, 14, ),
    (1, 3, 4, 8, 9, 10, 13, 19, ),
    (1, 2, 6, 7, 8, 12, 14, 19, ),
    (2, 3, 4, 5, 7, 13, 16, 17, ),
    (4, 5, 6, 9, 10, 12, 16, 18, ),
    (0, 1, 3, 4, 13, 15, 17, 18, ),
    (2, 8, 9, 10, 11, 14, 17, 19, ),
    (1, 4, 7, 11, 13, 14, 15, 16, ),
    (0, 2, 6, 7, 9, 10, 11, 12, ),
    (0, 2, 8, 9, 10, 11, 15, 19, ),
    (5, 7, 8, 9, 10, 16, 18, 19, ),
    (0, 2, 4, 5, 11, 13, 14, 18, ),
    (3, 6, 8, 11, 12, 15, 16, 19, )
]

goodStartingPoints  = []

# def genSolution():
#     global l
#     newSoltuion = np.zeros((l, 20), dtype=int)
#     for i in range(l):
#         newSoltuion[i,:] = drawMask()
#     return newSoltuion

for maskList in its.combinations(previousSolution,l):
    newSoltuion = np.zeros((l, 20), dtype=int)
    for i in range(l):
            newMask = np.zeros((1,20), dtype=int)
            for pos in maskList[i]:
                newMask[0,pos] = 1
            newSoltuion[i,:] = newMask
    goodStartingPoints.append(newSoltuion)


In [19]:
# Generate 'n' solutions
solutions = []

for i in range(n):
    # solutions.append(copy.deepcopy(goodStartingPoints[random.randrange(len(goodStartingPoints))]))
    solutions.append(genSolution())

In [20]:
# Score all solutions
scoredSolutions = []
for solution in solutions:
    scoredSolutions.append(scoreSolution(solution))

In [21]:
# # Testing between logical_and bitwise_and and matmul
# iter = 100000
# mask = drawMask()

# # logical_and
# start_time = time.time()
# for _ in range(iter):
#     matchCountColumn = np.logical_and(mask, ot).sum(axis=1)
#     detectionColumn1 = np.where(matchCountColumn < maskLen, 0, 1)
# print(f"--- logical_and took {time.time() - start_time}s total (avgerage {(time.time() - start_time)/iter} per iteration )---")


# # bitwise_and
# start_time = time.time()
# for _ in range(iter):
#     matchCountColumn = np.bitwise_and(mask, ot).sum(axis=1)
#     detectionColumn2 = np.where(matchCountColumn < maskLen, 0, 1)
# print(f"--- bitwise_and took {time.time() - start_time}s total (avgerage {(time.time() - start_time)/iter} per iteration )---")


# # matmul
# start_time = time.time()
# for _ in range(iter):
#     matchCountColumn = np.matmul(ot,mask)
#     detectionColumn3 = np.where(matchCountColumn < maskLen, 0, 1)
# print(f"--- matmul took {time.time() - start_time}s total (avgerage {(time.time() - start_time)/iter} per iteration )---")


In [22]:
# Sort the scores
scoredSolutions = sorted(scoredSolutions,key=itemgetter(0))
print(f"Cut off score: {scoredSolutions[s][0]}")
print(f"Best score {scoredSolutions[-1][0]}")

Cut off score: 4218
Best score 4372


In [23]:
def task(informationTuple):
    totalMutationAttmepts = 0
    definiteSuccess = 0
    mutatedCount = 0

    idx = random.randrange(informationTuple[1],informationTuple[2])
    newSolution = copy.deepcopy(informationTuple[1][idx])
    oldSolution = copy.deepcopy(informationTuple[1][idx])

    for _ in range(30):
        totalMutationAttmepts += 1

        newSolution = mutateElementDefinite(newSolution) 
        if (not np.array_equal(newSolution[2],oldSolution[2])):
            definiteSuccess += 1
        oldSolution = copy.deepcopy(newSolution)
        
    if (not np.array_equal(newSolution[2],informationTuple[1][idx][2])):
        mutatedCount = 1

    
    return ((mutatedCount, totalMutationAttmepts, definiteSuccess), newSolution)

In [24]:
def double(a):
    return a*2

In [25]:
# Search until we hit our goal score
generation = 0
while (scoredSolutions[-1][0] != goal):
    generation += 1
    totalMutationAttmepts = 0
    rowSuccess = 0
    elementSuccess = 0
    definiteSuccess = 0
    matrixSuccess = 0

    # Generate next generation
    mutatedCount = 0
    mutatedScores = []
    
    for i in range(n):
        idx = random.randrange(s,n)
        newSolution = copy.deepcopy(scoredSolutions[idx])
        oldSolution = copy.deepcopy(scoredSolutions[idx])
        for _ in range(l * maskLen):
            totalMutationAttmepts += 1

            # start_time = time.time()
            newSolution = mutateElementDefinite(newSolution) 
            # print("--- mutateElementDefinite took %s seconds ---" % (time.time() - start_time))
            if (not np.array_equal(newSolution[2],oldSolution[2])):
                definiteSuccess += 1
            oldSolution = copy.deepcopy(newSolution)
            
        mutatedScores.append(newSolution)
        if (not np.array_equal(newSolution[2],scoredSolutions[idx][2])):
            mutatedCount += 1

    # Sort solutions by score
    scoredSolutions = sorted(mutatedScores,key=itemgetter(0))

    # Print stats
    print(f"Generation {generation} complete")
    print(f"New cut off:\t\t{scoredSolutions[s][0]}/{goal}")
    print(f"Best Score:\t\t{scoredSolutions[-1][0]}/{goal}")
    print(f"Mutated:\t\t{round(mutatedCount/len(mutatedScores) * 100, 2)}%")
    # print(f"Row Mutation Rate:\t{round(rowSuccess/totalMutationAttmepts * 100, 2)}%")
    # print(f"Element Mutation Rate:\t{round(elementSuccess/totalMutationAttmepts * 100, 2)}%")
    print(f"Definite Mutation Rate:\t{round(definiteSuccess/totalMutationAttmepts * 100, 2)}%")

Generation 1 complete
New cut off:		4258/4845
Best Score:		4354/4845
Mutated:		100.0%
Definite Mutation Rate:	89.12%
Generation 2 complete
New cut off:		4285/4845
Best Score:		4350/4845
Mutated:		100.0%
Definite Mutation Rate:	88.11%
Generation 3 complete
New cut off:		4308/4845
Best Score:		4383/4845
Mutated:		100.0%
Definite Mutation Rate:	87.19%
Generation 4 complete
New cut off:		4327/4845
Best Score:		4398/4845
Mutated:		100.0%
Definite Mutation Rate:	86.0%
Generation 5 complete
New cut off:		4342/4845
Best Score:		4395/4845
Mutated:		100.0%
Definite Mutation Rate:	84.89%
Generation 6 complete
New cut off:		4354/4845
Best Score:		4406/4845
Mutated:		100.0%
Definite Mutation Rate:	83.85%
Generation 7 complete
New cut off:		4366/4845
Best Score:		4404/4845
Mutated:		100.0%
Definite Mutation Rate:	82.97%
Generation 8 complete
New cut off:		4376/4845
Best Score:		4428/4845
Mutated:		100.0%
Definite Mutation Rate:	82.01%
Generation 9 complete
New cut off:		4386/4845
Best Score:		4439/4

In [26]:
# Solution found
print("Solution found!")
with open("solution.txt", 'w') as outFile:
    for mask in scoredSolutions[-1][2]:
        ones = np.where(mask == 1)[0]
        outFile.write("(")
        for pos in ones:
            outFile.write(f"{str(pos)}, ")
        outFile.write(")\n")

Solution found!
